In [33]:
import keras
import numpy as np
import matplotlib.pyplot as plt

In [34]:
# function to display progress

def print_progress(percentage):
    percentage = int(percentage / 2)
    if percentage < 1:
        print("[{}]".format(" " * 50), end="\r")
    elif 0 < percentage < 50:
        print("[{}]".format("=" * (percentage - 1) + ">" + " " * (50 - percentage)), end="\r")
    else:
        print("[{}]".format("=" * 50))

# A

### Bilder, Modelle und MNIST Set laden

In [35]:
images = np.load("samples.npy")

model_one = keras.models.load_model("model1.hdf5")
model_two = keras.models.load_model("model2.hdf5")

(X_train, Y_train), (X_test, Y_test) = keras.datasets.mnist.load_data()

X = np.concatenate([X_train, X_test])
Y = np.concatenate([Y_train, Y_test])

### Original Labels aus dem MNIST Datenset extrahieren
Zuerst formatiere ich die Bilder aus der Datei etwas, damit ich sie mit den Bildern aus dem MNIST Datenset vergleichen
kann:
* entfernen der letzten Dimension
* mit 255 multiplizieren und zum `int` konvertieren

In [36]:
images_flat = (images[:, :, :, 0] * 255).astype(int)

labels = []

print("Labeling Pictures...")

for image in images_flat:
    print_progress(len(labels))
    found = False
    idx = 0
    for x in X:
        if np.array_equal(image, x):
            found = True
            labels.append(Y[idx])
            continue
        idx += 1
    if not found:
        raise ValueError("Could not determine Label of image")

print_progress(len(labels))

print("Done!")

Labeling Pictures...
[==================================================]
Done!


### Beide Modelle alle Bilder vorhersagen lassen

In [37]:
res_model_one = model_one.predict(images)
res_model_two = model_two.predict(images)

### Validierung der Vorhersagen
Regeln:
* Wenn nur eines von beiden Modellen die Zahl richtig vorhergesagt hat ordnen wir das Bild dem Modell zu
* Wenn beide das Bild richtig vorhergesagt haben wird das Bild dem Modell mit der höheren Confidence zugeordnet
* Wenn beide Modelle eine falsche Vorhersage gemacht hat wird das Bild nicht zugeordnet (mit '?' markiert)

In [38]:
result = []

for idx in range(len(images)):
    one_val_idx = np.argsort(res_model_one[idx])[len(res_model_one[idx]) - 1]
    two_val_idx = np.argsort(res_model_two[idx])[len(res_model_two[idx]) - 1]
    
    if one_val_idx != labels[idx] and two_val_idx != labels[idx]:
        result.append("?")
    elif one_val_idx == labels[idx] != two_val_idx:
        result.append(1)
    elif two_val_idx == labels[idx] != one_val_idx:
        result.append(2)
    else:
        result.append(1 if res_model_one[idx][one_val_idx] > res_model_two[idx][two_val_idx] else 2)

In [39]:
one = 0
two = 0
unmarked = 0

for res in result:
    if res == 1:
        one += 1
    elif res == 2:
        two += 1
    else:
        unmarked += 1

print("One: {}\nTwo: {}\nUnmarked: {}".format(one, two, unmarked))


One: 66
Two: 34
Unmarked: 0
